In [1]:
!pip install transformers datasets tokenizer seqeval

In [2]:
import datasets
import numpy as np
from transformers import BertTokenizerFast,DataCollatorForTokenClassification,AutoModelForTokenClassification

In [3]:
conll2003=datasets.load_dataset('conll2003')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
conll2003['train'][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [5]:
conll2003['train']

Dataset({
    features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
    num_rows: 14041
})

In [6]:
conll2003['train'].features['ner_tags']

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [7]:
conll2003['train'].description


'The shared task of CoNLL-2003 concerns language-independent named entity recognition. We will concentrate on\nfour types of named entities: persons, locations, organizations and names of miscellaneous entities that do\nnot belong to the previous three groups.\n\nThe CoNLL-2003 shared task data files contain four columns separated by a single space. Each word has been put on\na separate line and there is an empty line after each sentence. The first item on each line is a word, the second\na part-of-speech (POS) tag, the third a syntactic chunk tag and the fourth the named entity tag. The chunk tags\nand the named entity tags have the format I-TYPE which means that the word is inside a phrase of type TYPE. Only\nif two phrases of the same type immediately follow each other, the first word of the second phrase will have tag\nB-TYPE to show that it starts a new phrase. A word with tag O is not part of a phrase. Note the dataset uses IOB2\ntagging scheme, whereas the original dataset uses 

In [8]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [9]:
example_text = conll2003['train'][0]

In [10]:
example_text

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [11]:
tokenized_input = tokenizer(example_text['tokens'],is_split_into_words=True)

In [12]:
tokenized_input

{'input_ids': [101, 7327, 19164, 2446, 2655, 2000, 17757, 2329, 12559, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [13]:
token=tokenizer.convert_ids_to_tokens(tokenized_input['input_ids'])

In [14]:
token

['[CLS]',
 'eu',
 'rejects',
 'german',
 'call',
 'to',
 'boycott',
 'british',
 'lamb',
 '.',
 '[SEP]']

In [15]:
word_ids = tokenized_input.word_ids()
word_ids

[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, None]

In [16]:
def tokenize_and_align_label(examples,label_all_tokens=True):
  tokenized_input = tokenizer(examples['tokens'],truncation=True,is_split_into_words=True)
  labels=[]

  for i,label in enumerate(examples['ner_tags']):
    word_ids = tokenized_input.word_ids(batch_index=i)
    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:
      if word_idx is None:
        label_ids.append(-100)
      elif word_idx != previous_word_idx:
        label_ids.append(label[word_idx])
      else:
        label_ids.append(label[word_idx] if label_all_tokens else -100)
      previous_word_idx = word_idx

    labels.append(label_ids)

  tokenized_input['labels']= labels
  return tokenized_input


In [17]:
len(conll2003['train'][4]["tokens"])

31

In [18]:
q= tokenize_and_align_label(conll2003['train'][4:5])

In [20]:
for token,label in zip(tokenizer.convert_ids_to_tokens(q['input_ids'][0]),q['labels'][0]):
  print(f"{token:_<40}{label}")

[CLS]___________________________________-100
germany_________________________________5
'_______________________________________0
s_______________________________________0
representative__________________________0
to______________________________________0
the_____________________________________0
european________________________________3
union___________________________________4
'_______________________________________0
s_______________________________________0
veterinary______________________________0
committee_______________________________0
werner__________________________________1
z_______________________________________2
##wing__________________________________2
##mann__________________________________2
said____________________________________0
on______________________________________0
wednesday_______________________________0
consumers_______________________________0
should__________________________________0
buy_____________________________________0
sheep__________________________

In [21]:
tokenized_datasets = conll2003.map(tokenize_and_align_label,batched=True)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [22]:
tokenized_datasets['train'][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0],
 'input_ids': [101,
  7327,
  19164,
  2446,
  2655,
  2000,
  17757,
  2329,
  12559,
  1012,
  102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, -100]}

In [34]:
model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased",num_labels=9)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
from transformers import TrainingArguments,Trainer

In [36]:
!pip install transformers['torch'] accelerate -U

In [47]:
args = TrainingArguments(
"test-ner",
eval_strategy = "epoch",
learning_rate=2e-5,
per_device_train_batch_size=16,
per_device_eval_batch_size=16,
num_train_epochs=10,
weight_decay=0.01
)

In [48]:
example = conll2003['train'][0]

In [49]:
label_list = conll2003['train'].features['ner_tags'].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [50]:
!pip install evaluate

In [51]:
import evaluate

In [52]:
metric=evaluate.load("seqeval")

In [53]:
def compute_metrics(eval_preds):
    pred_logits, labels = eval_preds

    pred_logits = np.argmax(pred_logits, axis=2)
    # the logits and the probabilities are in the same order,
    # so we don’t need to apply the softmax

    # We remove all the values where the label is -100
    predictions = [
        [label_list[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(pred_logits, labels)
    ]

    true_labels = [
      [label_list[l] for (eval_preds, l) in zip(prediction, label) if l != -100]
       for prediction, label in zip(pred_logits, labels)
   ]
    results = metric.compute(predictions=predictions, references=true_labels)

    return {
          "precision": results["overall_precision"],
          "recall": results["overall_recall"],
          "f1": results["overall_f1"],
          "accuracy": results["overall_accuracy"],
  }

In [54]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [55]:
trainer=Trainer(
   model,
   args,
   train_dataset=tokenized_datasets["train"],
   eval_dataset=tokenized_datasets["validation"],
   data_collator=data_collator,
   processing_class=tokenizer,
   compute_metrics=compute_metrics
)

In [56]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.054700,0.058185,0.926556,0.932878,0.929706,0.983891
2,0.035500,0.063345,0.935019,0.943282,0.939132,0.985019
3,0.019300,0.063051,0.936302,0.945520,0.940888,0.985925
4,0.012600,0.067918,0.924869,0.950218,0.937372,0.985035
5,0.008000,0.071484,0.943153,0.948428,0.945783,0.986465
6,0.004700,0.076890,0.934687,0.947757,0.941176,0.986036
7,0.004100,0.075513,0.940472,0.949099,0.944766,0.986544
8,0.002500,0.081246,0.944759,0.950889,0.947814,0.986814
9,0.002100,0.084608,0.944130,0.950889,0.947497,0.986846
10,0.001600,0.082397,0.944623,0.952232,0.948412,0.987037


TrainOutput(global_step=8780, training_loss=0.014368101957854486, metrics={'train_runtime': 1727.7593, 'train_samples_per_second': 81.267, 'train_steps_per_second': 5.082, 'total_flos': 3405845692350414.0, 'train_loss': 0.014368101957854486, 'epoch': 10.0})

In [57]:
model.save_pretrained("ner_model")

In [58]:
tokenizer.save_pretrained('tokenizer')

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/vocab.txt',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [59]:
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [60]:
id2label = {
    str(i) : label for i,label in enumerate(label_list)
 }

In [61]:
id2label

{'0': 'O',
 '1': 'B-PER',
 '2': 'I-PER',
 '3': 'B-ORG',
 '4': 'I-ORG',
 '5': 'B-LOC',
 '6': 'I-LOC',
 '7': 'B-MISC',
 '8': 'I-MISC'}

In [62]:
label2id = {
    label : str(i) for i,label in enumerate(label_list)
}

In [63]:
label2id

{'O': '0',
 'B-PER': '1',
 'I-PER': '2',
 'B-ORG': '3',
 'I-ORG': '4',
 'B-LOC': '5',
 'I-LOC': '6',
 'B-MISC': '7',
 'I-MISC': '8'}

In [64]:
import json

In [65]:
config = json.load(open('/content/ner_model/config.json'))

In [66]:
config

{'architectures': ['BertForTokenClassification'],
 'attention_probs_dropout_prob': 0.1,
 'classifier_dropout': None,
 'gradient_checkpointing': False,
 'hidden_act': 'gelu',
 'hidden_dropout_prob': 0.1,
 'hidden_size': 768,
 'id2label': {'0': 'LABEL_0',
  '1': 'LABEL_1',
  '2': 'LABEL_2',
  '3': 'LABEL_3',
  '4': 'LABEL_4',
  '5': 'LABEL_5',
  '6': 'LABEL_6',
  '7': 'LABEL_7',
  '8': 'LABEL_8'},
 'initializer_range': 0.02,
 'intermediate_size': 3072,
 'label2id': {'LABEL_0': 0,
  'LABEL_1': 1,
  'LABEL_2': 2,
  'LABEL_3': 3,
  'LABEL_4': 4,
  'LABEL_5': 5,
  'LABEL_6': 6,
  'LABEL_7': 7,
  'LABEL_8': 8},
 'layer_norm_eps': 1e-12,
 'max_position_embeddings': 512,
 'model_type': 'bert',
 'num_attention_heads': 12,
 'num_hidden_layers': 12,
 'pad_token_id': 0,
 'position_embedding_type': 'absolute',
 'torch_dtype': 'float32',
 'transformers_version': '4.51.3',
 'type_vocab_size': 2,
 'use_cache': True,
 'vocab_size': 30522}

In [67]:
config['id2label'] = id2label

In [68]:
config['label2id'] = label2id

In [70]:
json.dump(config,open('/content/ner_model/config.json','w'))

In [78]:
model_fine_tuned = AutoModelForTokenClassification.from_pretrained('ner_model').to('cuda')

In [79]:
model_fine_tuned

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [80]:
from transformers import pipeline

In [81]:
nlp_pipeline = pipeline('ner',model_fine_tuned,tokenizer=tokenizer)

Device set to use cuda:0


In [82]:
nlp_pipeline

'EU rejects German call to boycott British lamb .'

In [95]:
example = " ".join(example_text['tokens'])

In [100]:
nlp_pipeline(example)

[{'entity': 'B-ORG',
  'score': np.float32(0.99981743),
  'index': 1,
  'word': 'eu',
  'start': 0,
  'end': 2},
 {'entity': 'B-MISC',
  'score': np.float32(0.9998301),
  'index': 3,
  'word': 'german',
  'start': 11,
  'end': 17},
 {'entity': 'B-MISC',
  'score': np.float32(0.9998405),
  'index': 7,
  'word': 'british',
  'start': 34,
  'end': 41}]

In [101]:
example_text

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}